# *GEO Maps in Python*

In [ ]:
import plotly.express as px

df = px.data.election()

geojson = px.data.election_geojson()

print(df["district"][2])
print(geojson["features"][0]["properties"])

In [ ]:
geojson

In [ ]:
df.head()

In [ ]:
fig = px.choropleth_mapbox(df, geojson=geojson, color="Coderre",
                           locations="district", featureidkey="properties.district",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import pandas as pd
# Sample population data
data = {
    'Country': ['China', 'India', 'United States', 'Indonesia', 'Pakistan'],
    'Population': [1444216107, 1393409038, 332915073, 276361783, 225199937]
}

# Create a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [ ]:
import plotly.express as px
# Create the map visualization
fig = px.choropleth(df, locations='Country', locationmode='country names', color = 'Population',
                    title='Global Population by Country')
# Display the map
fig.show()

# let's make a COVID 19 Map

In [ ]:
# import libraries
import pandas as pd
import plotly.express as p

In [ ]:
# import the data
df = pd.read_csv("D:\DS & ai\Data/covid-data.csv")
df.head()


In [ ]:
# set max column print option
pd.set_option('display.max_columns', None)

In [ ]:
df['total_cases'].value_counts()

In [ ]:
# lets take an average of total case by location using groupby function and create a new df
df_total_cases = df.groupby(['location'])['total_cases'].mean().reset_index()
df_total_cases

In [ ]:
# # Create the map visualization
# fig = px.choropleth(df_total_cases, locations='location', locationmode='country names', color = 'total_cases',
#                     title='Total Cases by Country', range_color=[0, 10000000])
# # Display the map
# fig.show()

In [ ]:
# px.choropleth(df, locations='location', locationmode='country names', color = 'new_cases',
#                     title='Total Cases by Country', range_color=[0, 10000],
#                     animation_frame= 'date'
#                     )

# Assignment: Animate this graph using Monthly Data, convert the data into monthly data and then animated the plot, save it as avi and mp4 file or gif

In [6]:
# group df based on each month in date and take average of new_cases
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Create a new column 'year_month' to represent the year and month
df['year_month'] = df['date'].dt.to_period('M')

# Now group by 'year_month' and 'location', and sum the 'new_cases' for each group
monthly_cases = df.groupby(['year_month', 'location']).agg({'total_cases': 'sum'}).reset_index()

# Convert 'year_month' back to a string format for readability
monthly_cases['year_month'] = monthly_cases['year_month'].astype(str)

# This will give you a new DataFrame with the sum of new cases for each month and location
monthly_cases.head()

,year_month,location,total_cases
0,2020-01,Afghanistan,0.0
1,2020-01,Africa,0.0
2,2020-01,Albania,0.0
3,2020-01,Algeria,0.0
4,2020-01,American Samoa,0.0


In [ ]:
# fig = px.ba(monthly_cases, locations='location', locationmode='country names', color = 'total_cases',
#                     title='Total Covid Cases Worldwide (January 2021 -to- April 2023)', 
#                     range_color=[0, monthly_cases['total_cases'].quantile(0.85)],
#                     animation_frame= 'year_month', color_continuous_scale='viridis',
#                     labels={'year_month':'Year-Month', 'total_cases':'Total Cases'}
#                     )
# # increase the size of the map
# fig.update_layout(height=600, width=800)


In [ ]:
df.head()

In [7]:
import plotly.express as px 
import io 
import PIL

fig = px.scatter( 
    monthly_cases, 
    x ="continent", 
    y ="total_cases", 
    animation_frame ="year_month", 
    animation_group ="continent", 
    size = "total_cases",
    range_y =[0, 9000000000] 
) 
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['year_month', 'location', 'total_cases'] but received: continent

In [ ]:
# saving animated gif
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL
fig = px.choropleth(monthly_cases, locations='location', locationmode='country names', color = 'total_cases',
                    title='Total Covid Cases Worldwide (January 2021 -to- April 2023)', 
                    range_color=[0, monthly_cases['total_cases'].quantile(0.85)],
                    animation_frame= 'year_month', color_continuous_scale='viridis',
                    labels={'year_month':'Year-Month', 'total_cases':'Total Cases'}
                    )
# increase the size of the map
fig.update_layout(height=600, width=800)

# generate images for each step in animation
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png", scale=3))))
    
# create animated GIF
frames[0].save(
        "./output/total_covid_cases_worldwide.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500, # milliseconds per frame
        loop=0, # infinite loop
        dither=None  # Turn off dithering
    )